# Báo cáo đồ án môn "Khoa học dữ liệu ứng dụng"

Nhóm 9:
1. 18120533 - Dương Đoàn Bảo Sơn - https://github.com/baosonhcmus
2. 18120498 - Nguyễn Thị Hồng Nhung - https://github.com/nhung-ng
3. 18120009 - Vương Gia Bảo - https://github.com/bao12012000
4. 18120543 - Trần Đại Tài - https://github.com/trandaitai327

Link thùng chứa Github của nhóm: [Link](https://github.com/trandaitai327/IEEE-CIS-Fraud-Detection)

## Mô tả bài toán

[IEEE-CIS Fraud Detection](https://www.kaggle.com/c/ieee-fraud-detection/overview)
### Bối cảnh:
Đây là cuộc thì được tổ chức bởi  Các nhà nghiên cứu từ IEEE Computational Intelligence Society (IEEE-CIS) hợp tác với công ty dịch vụ thanh toán hàng đầu thế giới - Vesta Corporation  nhằm tìm kiếm các giải pháp tốt nhất cho việc phát hiện gian lận từ các giao dịch của khách hàng, giúp hàng trăm nghìn doanh nghiệp giảm thất thoát do gian lận và tăng doanh thu của họ.
### Đánh giá:
Độ đo [area under the ROC curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)(AUC)
### Submission File:
Đối với mỗi `TransactionID` trong tập thử nghiệm, bạn phải dự đoán một xác suất cho biến `isFraud`. Tệp phải chứa tiêu đề và có định dạng sau:

TransactionID,isFraud

3663549,0.5

3663550,0.5

3663551,0.5

etc.

### Dữ liệu:
- Dữ liệu đến từ các giao dịch thương mại điện tử của các khách hàng của Vesta bao gồm các features từ loại thiết bị cho đến features của sản phẩm cũng như thông tin về các giao dịch và cho phép tạo các features mới để cải thiện kết quả của model.

- Dữ liệu được chia thành hai file:  identity và transactionđược nối với nhau bằng TransactionID. Không phải tất cả các giao dịch đều có thông tin nhận dạng tương ứng.

- Các thuộc tính:
  + Transaction Table:
    * TransactionDT: thời gian tính từ một móc tham chiếu  (ngày/ giờ) nhất định (không phải thời gian thực tế)
    * TransactionAMT: số tiền thanh toán cho giao dịch  (USD)
    * ProductCD:  mã sản phẩm  của mỗi giao dịch
    * card1 - card6: thông tin thẻ thanh toán, chẳng hạn như loại thẻ, ngân hàng phát hành,,quốc gia, v.v.
    * addr: địa chỉ user
    * dist: khoảng cách giữa (không giới hạn) địa chỉ thanh toán, địa chỉ gửi , mã zip, địa chỉ IP, mã vùng điện thoại, v.v.”
    * P_ and (R__) emaildomain: tên miền email của người mua và người nhận, một số giao dịch nhất định không cần người nhận, do đó, có thể null
    * C1-C14: counting  ,có bao nhiêu địa chỉ được tìm thấy có liên quan đến thẻ thanh toán, v.v. (Ý nghĩa thực tế được che giấu)
“Bạn có thể vui lòng cho thêm ví dụ về số đếm trong các biến C1-15 không? số điện thoại, địa chỉ email, tên liên quan đến người dùng?
    * D1-D15: timedelta (chẳng hạn thời gian giữa các lần giao dịch)
    * M1-M9: match ( tên trên thẻ , địa chỉ,vv..vv.)
    * Vxxx: các features  bao gồm xếp hạng, đếm và các quan hệ thực thể khác.

    “Ví dụ: số lần thẻ thanh toán được liên kết với IP và email hoặc địa chỉ xuất hiện trong phạm vi thời gian 24 giờ, v.v.” 
    * Categorical Features: ProductCD, card1 - card6, addr1, addr2, P_emaildomain, R_emaildomain, M1 - M9

  + Indentity Table:
    * Các biến trong bảng này là thông tin giúp xác thực/nhận dạng - thông tin kết nối mạng (IP, ISP, Proxy, v.v.) và chữ ký số (UA / browser / os / version, v.v.) được liên kết với các giao dịch.Chúng được thu thập bởi hệ thống chống gian lận của Vesta và các đối tác bảo mật kỹ thuật số.
(Các tên trường được che dấu và từ điển ghép nối sẽ không được cung cấp để bảo vệ quyền riêng tư và thỏa thuận hợp đồng) 
    * Categorical Features: DeviceType, DeviceInfo
    * id_12 - id_38 : là các numerical features cho Identity như rank thiết bị/ip_domain/ proxy, v.v. Ngoài ra, nó còn ghi lại dấu vân tay hành vi như số lần đăng nhập tài khoản / không đăng nhập được, thời gian đăng nhập tài khoản vẫn còn trên trang,.v..v

## Giải quyết bài toán

[Giải pháp hạng 1 - Chris Deotte & Konstantin Yakovlev](https://www.kaggle.com/c/ieee-fraud-detection/discussion/111284?fbclid=IwAR1CdMPVYUNmvIgxfDQbExwpYZZyU8SiMICVvfXwJEOSRIFEjE-2lUapRMg)   
**Time is not most important !!!**
- Không phải bản chất của gian lận thay đổi theo thời gian mà là do tập khách hàng thay đổi theo thời gian.
- Thử thách của cuộc thi này là dự đoán những khách hàng mà chúng ta không nhìn thấy được. Thực vậy dữ liệu private gồm 68.2% khách hàng chúng ta không nhìn thấy trong tập train, 16.4% tồn tại trong tập train và tập test, 15,4% là không chắc chắn (số liệu sau khi cuộc thi kết thúc)
### Tóm tắt cách giải quyết :
- Thay vì dự đoán giao dịch gian lận thì chúng ta phát hiện khách hàng gian lận.
#### 3 phần quan trọng trong solution :
- [EDA for Columns V and ID](https://www.kaggle.com/cdeotte/eda-for-columns-v-and-id)
- [Feature Engineering and Feature Selection](https://www.kaggle.com/c/ieee-fraud-detection/discussion/111308)
- [The Magic Works](https://www.kaggle.com/cdeotte/xgb-fraud-with-magic-0-9600#How-the-Magic-Works)

## EDA for Columns V and ID

### Prepare Data

In [ ]:
!gdown https://drive.google.com/uc?id=1x6xmQifUHQWZi7nDJs53do1G4q5aEnWI&export=download
!unzip ieee-fraud-detection.zip

### Import

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np,gc # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 500)

In [ ]:
cols_t = ['TransactionID', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain',
       'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8',
       'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4',
       'M5', 'M6', 'M7', 'M8', 'M9']
cols_v = ['V'+str(x) for x in range(1,340)]; types_v = {}
for c in cols_v: types_v[c] = 'float32'
train = pd.read_csv('train_transaction.csv',usecols=cols_t+['isFraud']+cols_v,dtype=types_v)

### NAN search
We will search all the columns of train_transaction.csv to determine which columns are related by the number of NANs present. We see that D1 relates to a subset of V281 thru V315, and D11 relates to V1 thru V11. Also we find groups of Vs with similar NAN structure. And we see that M1, M2, M3 related and M8, M9 related.

In [ ]:
nans_df = train.isna()
nans_groups={}
i_cols = ['V'+str(i) for i in range(1,340)]
for col in train.columns:
    cur_group = nans_df[col].sum()
    try:
        nans_groups[cur_group].append(col)
    except:
        nans_groups[cur_group]=[col]
del nans_df; x=gc.collect()

for k,v in nans_groups.items():
    print('####### NAN count =',k)
    print(v)

### V1-V11,D11

In [ ]:
Vc = ['dayr','isFraud','TransactionAmt','card1','addr1','D1n','D11n']
Vs = nans_groups[279287]
Vtitle = 'V1 - V11, D11'

In [ ]:
def make_plots(Vs):
    col = 4
    row = len(Vs)//4+1
    plt.figure(figsize=(20,row*5))
    idx = train[~train[Vs[0]].isna()].index
    for i,v in enumerate(Vs):
        plt.subplot(row,col,i+1)
        n = train[v].nunique()
        x = np.sum(train.loc[idx,v]!=train.loc[idx,v].astype(int))
        y = np.round(100*np.sum(train[v].isna())/len(train),2)
        t = 'int'
        if x!=0: t = 'float'
        plt.title(v+' has '+str(n)+' '+t+' and '+str(y)+'% nan')
        plt.yticks([])
        h = plt.hist(train.loc[idx,v],bins=100)
        if len(h[0])>1: plt.ylim((0,np.sort(h[0])[-2]))
    plt.show()
make_plots(Vs)

In [ ]:
def make_corr(Vs,Vtitle=''):
    cols = ['TransactionDT'] + Vs
    plt.figure(figsize=(15,15))
    sns.heatmap(train[cols].corr(), cmap='RdBu_r', annot=True, center=0.0)
    if Vtitle!='': plt.title(Vtitle,fontsize=14)
    else: plt.title(Vs[0]+' - '+Vs[-1],fontsize=14)
    plt.show()
make_corr(Vs,Vtitle)

In [ ]:
grps = [[1],[2,3],[4,5],[6,7],[8,9],[10,11]]
def reduce_group(grps,c='V'):
    use = []
    for g in grps:
        mx = 0; vx = g[0]
        for gg in g:
            n = train[c+str(gg)].nunique()
            if n>mx:
                mx = n
                vx = gg
            #print(str(gg)+'-'+str(n),', ',end='')
        use.append(vx)
        #print()
    print('Use these',use)
reduce_group(grps)

### V12 - V34

In [ ]:
Vs = nans_groups[76073]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[12,13],[14],[15,16,17,18,21,22,31,32,33,34],[19,20],[23,24],[25,26],[27,28],[29,30]]
reduce_group(grps)

### V35 - V52

In [ ]:
Vs = nans_groups[168969]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[35,36],[37,38],[39,40,42,43,50,51,52],[41],[44,45],[46,47],[48,49]]
reduce_group(grps)

### V53 - V74

In [ ]:
Vs = nans_groups[77096]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[53,54],[55,56],[57,58,59,60,63,64,71,72,73,74],[61,62],[65],[66,67],[68],[69,70]]
reduce_group(grps)

### V75 - V94

In [ ]:
Vs = nans_groups[89164]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[75,76],[77,78],[79,80,81,84,85,92,93,94],[82,83],[86,87],[88],[89],[90,91]]
reduce_group(grps)

### V95 - V137

In [ ]:
Vs = nans_groups[314]
make_corr(Vs)

In [ ]:
Vs = ['V'+str(x) for x in range(95,107)]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[95,96,97,101,102,103,105,106],[98],[99,100],[104]]
reduce_group(grps)

In [ ]:
Vs = ['V'+str(x) for x in range(107,124)]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[107],[108,109,110,114],[111,112,113],[115,116],[117,118,119],[120,122],[121],[123]]
reduce_group(grps)

In [ ]:
Vs = ['V'+str(x) for x in range(124,138)]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[124,125],[126,127,128,132,133,134],[129],[130,131],[135,136,137]]
reduce_group(grps)

### V138 ~ V163

In [ ]:
Vs = nans_groups[508595]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[138],[139,140],[141,142],[146,147],[148,149,153,154,156,157,158],[161,162,163]]
reduce_group(grps)

### V143 ~ V166

In [ ]:
Vs = nans_groups[508589]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[143,164,165],[144,145,150,151,152,159,160],[166]]
reduce_group(grps)

### V167 ~ V216

In [ ]:
Vs = [x for x in nans_groups[450909]]
make_corr(Vs)

In [ ]:
Vs = [x for x in nans_groups[450909] if int(x[1:])<186]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[167,168,177,178,179],[172,176],[173],[181,182,183]]
reduce_group(grps)

In [ ]:
Vs = [x for x in nans_groups[450909] if (int(x[1:])>183)]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[186,187,190,191,192,193,196,199],[202,203,204,211,212,213],[205,206],[207],[214,215,216]]
reduce_group(grps)

### V169 ~ V210

In [ ]:
Vs = nans_groups[450721]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[169],[170,171,200,201],[174,175],[180],[184,185],[188,189],[194,195,197,198],[208,210],[209]]
reduce_group(grps)

### V217 ~ V278

In [ ]:
Vs = [x for x in nans_groups[460110]]
make_corr(Vs)

In [ ]:
Vs = [x for x in nans_groups[460110] if int(x[1:])<240]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[217,218,219,231,232,233,236,237],[223],[224,225],[226],[228],[229,230],[235]]
reduce_group(grps)

In [ ]:
Vs = [x for x in nans_groups[460110] if (int(x[1:])>237)&(int(x[1:])<263)]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[240,241],[242,243,244,258],[246,257],[247,248,249,253,254],[252],[260],[261,262]]
reduce_group(grps)

In [ ]:
Vs = [x for x in nans_groups[460110] if (int(x[1:])>262)]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[263,265,264],[266,269],[267,268],[273,274,275],[276,277,278]]
reduce_group(grps)

### V220 ~ V272

In [ ]:
Vs = nans_groups[449124]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[220],[221,222,227,245,255,256,259],[234],[238,239],[250,251],[270,271,272]]
reduce_group(grps)

### V279 ~ V321

In [ ]:
Vs = [x for x in nans_groups[12]]
make_corr(Vs)

In [ ]:
Vs = [x for x in nans_groups[12] if int(x[1:])<302]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[279,280,293,294,295,298,299],[284],[285,287],[286],[290,291,292],[297]]
reduce_group(grps)

In [ ]:
Vs = [x for x in nans_groups[12] if int(x[1:])>299]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[302,303,304],[305],[306,307,308,316,317,318],[309,311],[310,312],[319,320,321]]
reduce_group(grps)

### V281 ~ V315, D1

In [ ]:
Vs = nans_groups[1269]
make_plots(Vs)
Vtitle = 'V281 - V315, D1'
make_corr(Vs,Vtitle)

In [ ]:
grps = [[281],[282,283],[288,289],[296],[300,301],[313,314,315]]
reduce_group(grps)

### V322 - V339

In [ ]:
Vs = nans_groups[508189]
make_plots(Vs)
make_corr(Vs)

In [ ]:
grps = [[322,323,324,326,327,328,329,330,331,332,333],[325],[334,335,336],[337,338,339]]
reduce_group(grps)

### V Reduced

In [ ]:
v =  [1, 3, 4, 6, 8, 11]
v += [13, 14, 17, 20, 23, 26, 27, 30]
v += [36, 37, 40, 41, 44, 47, 48]
v += [54, 56, 59, 62, 65, 67, 68, 70]
v += [76, 78, 80, 82, 86, 88, 89, 91]
v += [96, 98, 99, 104]
v += [107, 108, 111, 115, 117, 120, 121, 123]
v += [124, 127, 129, 130, 136]
v += [138, 139, 142, 147, 156, 162]
v += [165, 160, 166]
v += [178, 176, 173, 182]
v += [187, 203, 205, 207, 215]
v += [169, 171, 175, 180, 185, 188, 198, 210, 209]
v += [218, 223, 224, 226, 228, 229, 235]
v += [240, 258, 257, 253, 252, 260, 261]
v += [264, 266, 267, 274, 277]
v += [220, 221, 234, 238, 250, 271]
v += [294, 284, 285, 286, 291, 297]
v += [303, 305, 307, 309, 310, 320]
v += [281, 283, 289, 296, 301, 314]
v += [332, 325, 335, 338]

In [ ]:
print('Reduced set has',len(v),'columns')

In [ ]:
cols = ['TransactionDT'] + ['V'+str(x) for x in v]
train2 = train[cols].sample(frac=0.2)
plt.figure(figsize=(15,15))
sns.heatmap(train2[cols].corr(), cmap='RdBu_r', annot=False, center=0.0)
plt.title('V1-V339 REDUCED',fontsize=14)
plt.show()

### V All
We notice that the first 100 Vs don't correlate much with the last 239 Vs. Also even though the first 100 have 6 different NAN groups there is much correlation between the groups. The latter 239 relate to each other.

In [ ]:
cols = ['TransactionDT'] + ['V'+str(x) for x in range(1,340)]
train2 = train[cols].sample(frac=0.2)
plt.figure(figsize=(15,15))
sns.heatmap(train2[cols].corr(), cmap='RdBu_r', annot=False, center=0.0)
plt.title('V1-V339 ALL',fontsize=14)
plt.show()

### C Columns

In [ ]:
def make_plots2(Vs):
    col = 4
    row = len(Vs)//4+1
    plt.figure(figsize=(20,row*5))
    for i,v in enumerate(Vs):
        plt.subplot(row,col,i+1)
        idx = train[~train[v].isna()].index
        n = train[v].nunique()
        x = np.sum(train.loc[idx,v]!=train.loc[idx,v].astype(int))
        y = np.round(100*np.sum(train[v].isna())/len(train),2)
        t = 'int'
        if x!=0: t = 'float'
        plt.title(v+' has '+str(n)+' '+t+' and '+str(y)+'% nan')
        plt.yticks([])
        h = plt.hist(train.loc[idx,v],bins=100)
        if len(h[0])>1: plt.ylim((0,np.sort(h[0])[-2]))
    plt.show()
make_plots2(['C'+str(x) for x in range(1,15)])

In [ ]:
cols = ['TransactionDT'] + ['C'+str(x) for x in range(1,15)]
plt.figure(figsize=(15,15))
sns.heatmap(train[cols].corr(), cmap='RdBu_r', annot=True, center=0.0)
plt.title('C1-C15')
plt.show()

### D Columns

In [ ]:
cols = ['TransactionDT'] + ['D'+str(x) for x in range(1,16)]
plt.figure(figsize=(15,15))
sns.heatmap(train[cols].corr(), cmap='RdBu_r', annot=True, center=0.0)
plt.title('D1-D15')
plt.show()

### M Columns

In [ ]:
Ms = ['M'+str(x) for x in range(1,10)]
mp = {'F':0,'T':1,'M0':0,'M1':1,'M2':2}
for c in Ms: train[c] = train[c].map(mp)

In [ ]:
cols = ['TransactionDT'] + Ms
plt.figure(figsize=(15,15))
sns.heatmap(train[cols].corr(), cmap='RdBu_r', annot=True, center=0.0)
plt.title('M1-M9')
plt.show()

### ID Columns

In [ ]:
train_id = pd.read_csv('train_identity.csv')
train_id = pd.merge(train_id,train[['TransactionID','TransactionDT']],on='TransactionID',how='left')
ids = ['id_0'+str(x) for x in range(1,10)]+['id_'+str(x) for x in range(10,39)]
for c in ids: print (c,train_id[c].unique()[:10])

In [ ]:
booln = ['id_12','id_15','id_16','id_27','id_28','id_29','id_35','id_36','id_37','id_38']
cats = ['id_23','id_30','id_31','id_33','id_34']
mp = {'Unknown':0,'NotFound':1,'Found':2,'New':3,'F':0,'T':1}
for c in booln: train_id[c] = train_id[c].map(mp)

In [ ]:
def make_plots2(Vs):
    col = 4
    row = len(Vs)//4+1
    plt.figure(figsize=(20,row*5))
    for i,v in enumerate(Vs):
        plt.subplot(row,col,i+1)
        idx = train_id[~train_id[v].isna()].index
        n = train_id[v].nunique()
        x = np.sum(train_id.loc[idx,v]!=train_id.loc[idx,v].astype(int))
        y = np.round(100*np.sum(train_id[v].isna())/len(train_id),2)
        t = 'int'
        if x!=0: t = 'float'
        plt.title(v+' has '+str(n)+' '+t+' and '+str(y)+'% nan')
        plt.yticks([])
        h = plt.hist(train_id.loc[idx,v],bins=100)
        if len(h[0])>1: plt.ylim((0,np.sort(h[0])[-2]))
    plt.show()
make_plots2([x for x in ids if x not in cats])

In [ ]:
cols = ['TransactionDT'] + [x for x in ids if x not in cats]
plt.figure(figsize=(15,15))
sns.heatmap(train_id[cols].corr(), cmap='RdBu_r', annot=True, center=0.0)
plt.title('ID1-ID38')
plt.show()

 ## Feature Engineering and Feature Selection

Dưới đây là một số ý tưởng về cách tạo mới và chọn thuộc tính cho mô hình mà nhóm tác giả đã thực hiện:
>Kernel https://www.kaggle.com/kyakovlev/ieee-fe-for-local-test của Konstantin thực hiện các FE for local test và chứng minh các kỹ thuật đó. Mọi người có thể xem để tham khảo thêm
1. Tạo một feature mới và sau đó đánh giá nó bằng các tiến hành xác thực cục bộ local validation scheme  để xem liệu nó có cải thiện CV của mô hình hay không. Giữ lại các feature có lợi và loại bỏ các feature khác.
2. Lựa chọn các thuộc tính chuyển tiếp (sử dụng trên một thuộc tính hoặc nhóm các thuộc tính)
3. Loại bỏ thuộc tính đệ quy (sử dụng cho một thuộc tính hoặc nhóm các thuộc tính)
4. Xác đinh tầm quan trọng của thuộc tính bằng cách thực hiện hoán vị các giá trị thuộc tính.
5. Xác thực chéo.
6. Phân tích tương quan các thuộc tính.
7. Tính nhất quán về thời gian/
8. Tính nhất quán của khách hàng.
9. Kiểm tra sự phân phối của tập dữ liệu thử nghiệm so với tập dữ liệu đào tạo và nếu có thể xem chúng khác nhau như thế nào.

### Train & Test
Khi thực hiện mã hóa nhãn, phải mã hóa trên cùng tập dữ liệu Train và Test bằng cách gộp chung dữ liệu lại với nhau.   
```python
df = pd.concat([train[col],test[col]],axis=0) 
# PERFORM FEATURE ENGINEERING HERE
train[col] = df[:len(train)]   
test[col] = df[len(train):]
```


### NAN Processing
Nếu ta cung cấp giá trị np.nan cho LGBM/XGB, thì tại mỗi lần Tree phân chia node, nó sẽ tách các giá trị không phải NAN và sau đó gửi tất cả các NAN đến con trái hoặc con phải tùy thuộc vào điều gì tốt nhất. Do đó, các giá trị NAN được đối xử đặc biệt ở mọi node và có thể trở dẫn đến overfit.

-->Chuyển đổi tất cả NAN thành một số âm thấp hơn tất cả các giá trị không phải NAN (chẳng hạn như - 999) thì mô hình sẽ không ưu tiên xử lý với NAN nữa và dành cho nó sự chú ý giống như những con số khác.Trong bài này tác giả sử dụng giá trị -1   
```python
df[col].fillna(-999, inplace=True)
```

### Label Encode/ Factorize/ Memory reduction
Factorizing -Mã hóa nhãn : chuyển đổi một cột kiểu string, category, object thành integers. Sau đó, ta có thể chuyển thành kiểu int8, int16 hoặc int32 tùy thuộc vào việc kiểm tra giá trị lớn nhất của cột. Từ đó giúp giảm bộ nhớ lưu trữ.
```python
df[col],_ = df[col].factorize()
if df[col].max()<128: df[col] = df[col].astype('int8')
elif df[col].max()<32768: df[col] = df[col].astype('int16')
else: df[col].astype('int32')
```

Ngoài ra để giảm bộ nhớ, ta sử dụng memory_reducer trên các cột. Một cách tiếp cận đơn giản và an toàn hơn là chuyển đổi tất cả các giá trị kiểu float64 thành float32 và tất cả giá trị kiểu int64 thành int32. (Tốt nhất nên tránh float16 , sử dụng int8 và int16 nếu muốn).
```python
for col in df.columns:
    if df[col].dtype=='float64': df[col] = df[col].astype('float32')
    if df[col].dtype=='int64': df[col] = df[col].astype('int32')
```


### Categorical Features
Với các biến phân loại, ta có thể lựa chọn cho mô hình khi biết rằng chúng vó kiểu 'categorical' hoặc có thể coi nó là kiểu 'numeric' nếu ta đã mã hóa các nhãn của nó trước đó. Dù bằng cách nào, mô hình cũng có thể trích xuất các categorical classes.

### Splitting
Một cột kiểu string/numeric có thể được tách thành hai cột mới.

>Ví dụ, một cột feature kiểu string chẳng hạn như cột 'id_30' có gí trị “ Mac OS X 10_9_5”  có thể được chia thành 2 cột mới là:
>1. "Operating System"  có giá trị "Mac OS X” 
>2. "Version" có giá trị là  “10_9_5”. 
>
>Hoặc ví dụ với cột feature là "TransactionAmt" kiểu numeric có giá trị “ 1230.45” có thể được tách thành 2 cột ( tức là 2 feature mới) :
>1. "USD" có giá trị  “ 1230” 
>2. "Cent" có giá trị  “ 45”


### Combining / Transforming / Interaction
Hai cột có kiểu string/ numeric có thể được kết hợp thành một cột mới. 

>Ví dụ cột feature "card1" và feature "card2" có thể trở thành một cột feature mới là "uid"
```python
df['uid'] = df[‘card1’].astype(str)+’_’+df[‘card2’].astype(str)
```

>Điều này giúp ích cho mô hình bởi vì bản thân thuộc tính 'card1' và 'card2' có thể không tương quan với target của bài toán ,do đó mô hình sẽ không phân chia các thuộc tính này. Nhưng thuộc tính 'uid = card1_card2' có thể tương quan với target và bây giờ mô hình có thể sử dụng thuộc tính này.
>
> Các cột thuộc tính kiểu số - numeric có thể được kết hợp với nhau bằng các phép cộng, trừ, nhân, v.v. 
>
>ví dụ:
```python
 df['x1_x2'] = df['x1'] * df['x2']
```

### Frequency Encoding
>Mã hóa tần số là một kỹ thuật mạnh mẽ cho phép mô hình "nhìn thấy" liệu các giá trị nào đó của thuộc tính là hiếm gặp hay phổ biến. 
>
>Ví dụ: nếu muốn mô hình nhân ra thuộc tính "credit_card" nào được sử dụng thường xuyên t có thể tiến hành đếm tần số xuất hiện của tất cả các giá trị cột "card1":
```python
temp = df['card1'].value_counts().to_dict()
df['card1_counts'] = df['card1'].map(temp)
```

### Aggregations / Group Statistics
>Cung cấp cho mô hình các thông kê giá trị theo nhóm thuộc tính cho phép mô hình xác định xem giá trị nào là phổ biến hay hiếm gặp đối với từng nhóm thuộc tính cụ thể.
```python
temp = df.groupby('card1')['TransactionAmt'].agg(['mean'])   
    .rename({'mean':'TransactionAmt_card1_mean'},axis=1)
df = pd.merge(df,temp,on='card1',how='left')
```

### Normalize / Standardize
>Chuẩn hóa các cột so với chính chúng. Hoặc chuẩn hóa cột này so với một cột khác.
```python
#themselfs 
df[col] = ( df[col]-df[col].mean() ) / df[col].std()

#normalize D3
df['D3_remove_time'] = df['D3'] - df['D3_week_mean']
```

### Outlier Removal / Relax / Smooth / PCA
>Trong cuộc thi này, tác giả muốn tìm ra điểm bất thường nên tg sử dụng kỹ thuật làm mịn một cách cẩn thận.
>
>Ý tưởng : xác định và thay thế các giá trị không phổ biến.
>
>Ví dụ: bằng cách sử dụng mã hóa tần suất cho một biến, ta có thể loại bỏ tất cả các giá trị xuất hiện dưới 0,1% bằng cách thay thế chúng bằng một giá trị mới như -9999. (khác giá trị thay cho NAN)

## XGB Fraud
Trong kernel này, nhóm tg xây dựng hai mô hình XGB. Mô hình đầu tiên không sử dụng các magic feature và đạt được LB 0,95. Mô hình thứ hai sử dụng các magic feature và đạt được LB 0,96. Trong phần phụ lục, nhóm tg trình bày cách tăng LB hơn nữa với post processing.

### Load Data
>Nhóm tg load tất cả data ngoại trừ 229 V columns đã được tác giả xác định dư thừa thông qua phân tích tương quan như ở trên.

In [ ]:
BUILD95 = True
BUILD96 = True

import numpy as np, pandas as pd, os, gc
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

# COLUMNS WITH STRINGS
str_type = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain','M1', 'M2', 'M3', 'M4','M5',
            'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 
            'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']
str_type += ['id-12', 'id-15', 'id-16', 'id-23', 'id-27', 'id-28', 'id-29', 'id-30', 
            'id-31', 'id-33', 'id-34', 'id-35', 'id-36', 'id-37', 'id-38']

# FIRST 53 COLUMNS
cols = ['TransactionID', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain',
       'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8',
       'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4',
       'M5', 'M6', 'M7', 'M8', 'M9']

# V COLUMNS TO LOAD DECIDED BY CORRELATION EDA

cols += ['V'+str(x) for x in v]
dtypes = {}
for c in cols+['id_0'+str(x) for x in range(1,10)]+['id_'+str(x) for x in range(10,34)]+\
    ['id-0'+str(x) for x in range(1,10)]+['id-'+str(x) for x in range(10,34)]:
        dtypes[c] = 'float32'
for c in str_type: dtypes[c] = 'category'

In [ ]:
%%time
# LOAD TRAIN
X_train = pd.read_csv('train_transaction.csv',index_col='TransactionID', dtype=dtypes, usecols=cols+['isFraud'])
train_id = pd.read_csv('train_identity.csv',index_col='TransactionID', dtype=dtypes)
X_train = X_train.merge(train_id, how='left', left_index=True, right_index=True)
# LOAD TEST
X_test = pd.read_csv('test_transaction.csv',index_col='TransactionID', dtype=dtypes, usecols=cols)
test_id = pd.read_csv('test_identity.csv',index_col='TransactionID', dtype=dtypes)
fix = {o:n for o, n in zip(test_id.columns, train_id.columns)}
test_id.rename(columns=fix, inplace=True)
X_test = X_test.merge(test_id, how='left', left_index=True, right_index=True)
# TARGET
y_train = X_train['isFraud'].copy()
del train_id, test_id, X_train['isFraud']; x = gc.collect()
# PRINT STATUS
print('Train shape',X_train.shape,'test shape',X_test.shape)

### Chuẩn hóa các cột D
Công thức chuẩn hóa cho các giá trị cột D như sau:

 `D15n = Transaction_Day - D15`
 
 `Transaction_Day = TransactionDT/(24*60*60)`. 
 
D Columns là các khoảng thời gian được tính từ 1 thời điểm nào đó trong quá khứ. Nhóm tiến hành transform các cột D thành các ngày cụ thể trong quá khứ. Loại bỏ sự phụ thuộc theo thời gian (dây ko còn là bài toán time series) giá trị của các features D không còn tăng dần theo thời gian nữa.

In [ ]:
# PLOT ORIGINAL D
plt.figure(figsize=(15,5))
plt.scatter(X_train.TransactionDT,X_train.D15)
plt.title('Original D15')
plt.xlabel('Time')
plt.ylabel('D15')
plt.show()

In [ ]:
# NORMALIZE D COLUMNS
for i in range(1,16):
    if i in [1,2,3,5,9]: continue
    X_train['D'+str(i)] =  X_train['D'+str(i)] - X_train.TransactionDT/np.float32(24*60*60)
    X_test['D'+str(i)] = X_test['D'+str(i)] - X_test.TransactionDT/np.float32(24*60*60)

In [ ]:
# PLOT TRANSFORMED D
plt.figure(figsize=(15,5))
plt.scatter(X_train.TransactionDT,X_train.D15)
plt.title('Transformed D15')
plt.xlabel('Time')
plt.ylabel('D15n')
plt.show()

In [ ]:
%%time
# LABEL ENCODE AND MEMORY REDUCE
for i,f in enumerate(X_train.columns):
    # FACTORIZE CATEGORICAL VARIABLES
    if (np.str(X_train[f].dtype)=='category')|(X_train[f].dtype=='object'): 
        df_comb = pd.concat([X_train[f],X_test[f]],axis=0) #nối
        df_comb,_ = df_comb.factorize(sort=True)   # xáo trôn thứ tự
        if df_comb.max()>32000: print(f,'needs int32')
        X_train[f] = df_comb[:len(X_train)].astype('int16')
        X_test[f] = df_comb[len(X_train):].astype('int16')
    # SHIFT ALL NUMERICS POSITIVE. SET NAN to -1
    elif f not in ['TransactionAmt','TransactionDT']:
        mn = np.min((X_train[f].min(),X_test[f].min()))
        X_train[f] -= np.float32(mn)
        X_test[f] -= np.float32(mn)
        X_train[f].fillna(-1,inplace=True)
        X_test[f].fillna(-1,inplace=True)

### Encoding Functions
1. `encode_FE` : hàm này thực hiện frequency encode- mã hóa tần suất cho giá trị của các thuộc tính, trong đó kết hợp dữ liệu train và test trước khi thực hiện mã hóa. 
2. `encode_LE` :mã hóa nhãn cho các thuộc tính phân loại 
3.`encode_AG` : Tạo các thuộc tính tổng hợp ( theo giá trị trung bình hoặc độ lệch chẩun) bằng cách thống kê theo nhóm thuộc tính.
4. `encode_CB` :tạo new_feauturcác thuộc tính mới cho mô hình bằng cách kết hợp các thuộc tính lại với nhau. 
5.`encode_AG2`:tạo các thuộc tính tổng hợp duy nhất - thống kê số các giá trị khác nhau của mỗi nhóm.


In [ ]:
# FREQUENCY ENCODE TOGETHER
def encode_FE(df1, df2, cols):
    for col in cols:
        df = pd.concat([df1[col],df2[col]])
        vc = df.value_counts(dropna=True, normalize=True).to_dict()
        vc[-1] = -1
        nm = col+'_FE'
        df1[nm] = df1[col].map(vc)
        df1[nm] = df1[nm].astype('float32')
        df2[nm] = df2[col].map(vc)
        df2[nm] = df2[nm].astype('float32')
        print(nm,', ',end='')
        
# LABEL ENCODE
def encode_LE(col,train=X_train,test=X_test,verbose=True):
    df_comb = pd.concat([train[col],test[col]],axis=0)
    df_comb,_ = df_comb.factorize(sort=True)
    nm = col
    if df_comb.max()>32000: 
        train[nm] = df_comb[:len(train)].astype('int32')
        test[nm] = df_comb[len(train):].astype('int32')
    else:
        train[nm] = df_comb[:len(train)].astype('int16')
        test[nm] = df_comb[len(train):].astype('int16')
    del df_comb; x=gc.collect()
    if verbose: print(nm,', ',end='')
        
# GROUP AGGREGATION MEAN AND STD
# https://www.kaggle.com/kyakovlev/ieee-fe-with-some-eda
def encode_AG(main_columns, uids, aggregations=['mean'], train_df=X_train, test_df=X_test, 
              fillna=True, usena=False):
    # AGGREGATION OF MAIN WITH UID FOR GIVEN STATISTICS #thống kê theo uid
    for main_column in main_columns: 
        for col in uids:
            for agg_type in aggregations:
                new_col_name = main_column+'_'+col+'_'+agg_type
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]]) #concat new feature
                if usena: temp_df.loc[temp_df[main_column]==-1,main_column] = np.nan #thay nan vào các gt=-1 (đã đc xử lí trước đó)
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})  

                temp_df.index = list(temp_df[col]) #set index by uid
                temp_df = temp_df[new_col_name].to_dict() # convert to dict 

                train_df[new_col_name] = train_df[col].map(temp_df).astype('float32')  # map mean_value
                test_df[new_col_name]  = test_df[col].map(temp_df).astype('float32')
                
                if fillna:
                    train_df[new_col_name].fillna(-1,inplace=True)
                    test_df[new_col_name].fillna(-1,inplace=True)
                
                print("'"+new_col_name+"'",', ',end='')
                
# COMBINE FEATURES (create newfeaturé from 2 colums, then encode label for new_feature)
def encode_CB(col1,col2,df1=X_train,df2=X_test):
    nm = col1+'_'+col2
    df1[nm] = df1[col1].astype(str)+'_'+df1[col2].astype(str)
    df2[nm] = df2[col1].astype(str)+'_'+df2[col2].astype(str) 
    encode_LE(nm,verbose=False)
    print(nm,', ',end='')
    
# GROUP AGGREGATION NUNIQUE
def encode_AG2(main_columns, uids, train_df=X_train, test_df=X_test):
    for main_column in main_columns:  
        for col in uids:
            comb = pd.concat([train_df[[col]+[main_column]],test_df[[col]+[main_column]]],axis=0)
            mp = comb.groupby(col)[main_column].agg(['nunique'])['nunique'].to_dict()
            train_df[col+'_'+main_column+'_ct'] = train_df[col].map(mp).astype('float32')
            test_df[col+'_'+main_column+'_ct'] = test_df[col].map(mp).astype('float32')
            print(col+'_'+main_column+'_ct, ',end='')

### Feature Engineering
Tất cả các features này được chọn bởi vì mỗi features làm tăng giá trị local validation.

Quy trình:Đầu tiên tạo một feature mới. Sau đó, thêm nó vào mô hình của mình và đánh giá xem AUC tăng hay giảm. Nếu AUC tăng, giữ lại , nếu không thì loại bỏ.

In [ ]:
%%time
# TRANSACTION AMT CENTS #làm tròn
X_train['cents'] = (X_train['TransactionAmt'] - np.floor(X_train['TransactionAmt'])).astype('float32') 
X_test['cents'] = (X_test['TransactionAmt'] - np.floor(X_test['TransactionAmt'])).astype('float32')
print('cents, ', end='')
# FREQUENCY ENCODE: ADDR1, CARD1, CARD2, CARD3, P_EMAILDOMAIN #mã hóa tần suất
encode_FE(X_train,X_test,['addr1','card1','card2','card3','P_emaildomain'])
# COMBINE COLUMNS CARD1+ADDR1, CARD1+ADDR1+P_EMAILDOMAIN
encode_CB('card1','addr1')
encode_CB('card1_addr1','P_emaildomain')
# FREQUENCY ENOCDE
encode_FE(X_train,X_test,['card1_addr1','card1_addr1_P_emaildomain'])
# GROUP AGGREGATE
encode_AG(['TransactionAmt','D9','D11'],['card1','card1_addr1','card1_addr1_P_emaildomain'],['mean','std'],usena=True)

### Feature Selection - Time Consistency

Nhóm tg đã thêm 28 features mới ở trên. Vì vậy,hiện tại chúng ta có 242 features. Bây giờ ta sẽ kiểm tra từng feature để theo dõi tính nhất quán về thời gian của thuộc tính đó.
Nhóm tg xây dựng 242 mô hình- mỗi mô hình tương ứng với 1 thuộc tính.
Mỗi mô hình sẽ được đào tạo vào tháng đầu tiên của dữ liệu đào tạo và sẽ chỉ sử dụng một feature duy nhất. Sau đó, dự đoán kết quả trên tháng cuối cùng của dữ liệu đào tạo. Tác giả muốn cả AUC trên tập train và validation đều phải đạt trên 0,5. 
Sau khi hoàn thành 242 mô hình trên, tác giả chỉ ra rằng có 19 features không thành công trong thử nghiệm này, vì vậy ta sẽ loại bỏ chúng. Ngoài ra, tác giả cũng loại bỏ 7 cột D chủ yếu là NAN.

In [ ]:
cols = list( X_train.columns )
cols.remove('TransactionDT') 
for c in ['D6','D7','D8','D9','D12','D13','D14']:
    cols.remove(c) # a lot nan
    
# FAILED TIME CONSISTENCY TEST
for c in ['C3','M5','id_08','id_33']:
    cols.remove(c)
for c in ['card4','id_07','id_14','id_21','id_30','id_32','id_34']:
    cols.remove(c)
for c in ['id_'+str(x) for x in range(22,28)]:
    cols.remove(c)

In [ ]:
print('NOW USING THE FOLLOWING',len(cols),'FEATURES.')
np.array(cols)

### Xác thực chéo Local Validation
xác thực dựa trên thời gian. Tg đã đánh giá các thuộc tính bằng cách huấn luyện mô hình trên 75% dữ liệu đầu tiên và dự đoán trên 25% dữ liệu cuối cùng. 
Mô hình XGB của tác giả với 216 features đạt được giá trị AUC = 0,9363 trên tập validation. 

[1]: https://www.kaggle.com/kyakovlev/ieee-fe-for-local-test
[2]: https://www.kaggle.com/kyakovlev/ieee-lgbm-with-groupkfold-cv

In [ ]:
# CHRIS - TRAIN 75% PREDICT 25%
idxT = X_train.index[:3*len(X_train)//4]
idxV = X_train.index[3*len(X_train)//4:]

# KONSTANTIN - TRAIN 4 SKIP 1 PREDICT 1 MONTH
#idxT = X_train.index[:417559]
#idxV = X_train.index[-89326:]

In [ ]:
import xgboost as xgb
print("XGBoost version:", xgb.__version__)

if BUILD95:
    clf = xgb.XGBClassifier( 
        n_estimators=2000,
        max_depth=12, 
        learning_rate=0.02, 
        subsample=0.8,
        colsample_bytree=0.4, 
        missing=-1, 
        eval_metric='auc',
        # USE CPU
        #nthread=4,
        #tree_method='hist' 
        # USE GPU
        tree_method='gpu_hist' 
    )
    h = clf.fit(X_train.loc[idxT,cols], y_train[idxT], 
        eval_set=[(X_train.loc[idxV,cols],y_train[idxV])],
        verbose=50, early_stopping_rounds=100)

In [ ]:
if BUILD95:

    feature_imp = pd.DataFrame(sorted(zip(clf.feature_importances_,cols)), columns=['Value','Feature'])
    plt.figure(figsize=(20, 10))
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False).iloc[:50])
    plt.title('XGB95 Most Important Features')
    plt.tight_layout()
    plt.show()
    del clf, h; x=gc.collect()

### Predict test.csv

In [ ]:
import datetime
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
X_train['DT_M'] = X_train['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
X_train['DT_M'] = (X_train['DT_M'].dt.year-2017)*12 + X_train['DT_M'].dt.month 

X_test['DT_M'] = X_test['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
X_test['DT_M'] = (X_test['DT_M'].dt.year-2017)*12 + X_test['DT_M'].dt.month 

In [ ]:
if BUILD95:
    oof = np.zeros(len(X_train))
    preds = np.zeros(len(X_test))

    skf = GroupKFold(n_splits=6)
    for i, (idxT, idxV) in enumerate( skf.split(X_train, y_train, groups=X_train['DT_M']) ):
        month = X_train.iloc[idxV]['DT_M'].iloc[0]
        print('Fold',i,'withholding month',month)
        print(' rows of train =',len(idxT),'rows of holdout =',len(idxV))
        clf = xgb.XGBClassifier(
            n_estimators=5000,
            max_depth=12,
            learning_rate=0.02,
            subsample=0.8,
            colsample_bytree=0.4,
            missing=-1,
            eval_metric='auc',
            # USE CPU
            #nthread=4,
            #tree_method='hist'
            # USE GPU
            tree_method='gpu_hist' 
        )        
        h = clf.fit(X_train[cols].iloc[idxT], y_train.iloc[idxT], 
                eval_set=[(X_train[cols].iloc[idxV],y_train.iloc[idxV])],
                verbose=100, early_stopping_rounds=200)
    
        oof[idxV] += clf.predict_proba(X_train[cols].iloc[idxV])[:,1]
        preds += clf.predict_proba(X_test[cols])[:,1]/skf.n_splits
        del h, clf
        x=gc.collect()
    print('#'*20)
    print ('XGB95 OOF CV=',roc_auc_score(y_train,oof))

In [ ]:
if BUILD95:
    plt.hist(oof,bins=100)
    plt.ylim((0,5000))
    plt.title('XGB OOF')
    plt.show()

    X_train['oof'] = oof
    X_train.reset_index(inplace=True)
    X_train[['TransactionID','oof']].to_csv('oof_xgb_95.csv')
    X_train.set_index('TransactionID',drop=True,inplace=True)
    
else: X_train['oof'] = 0

### Kaggle Submission File XGB_95

In [ ]:
if BUILD95:
    sample_submission = pd.read_csv('sample_submission.csv')
    sample_submission.isFraud = preds
    sample_submission.to_csv('sub_xgb_95.csv',index=False)

    plt.hist(sample_submission.isFraud,bins=100)
    plt.ylim((0,5000))
    plt.title('XGB95 Submission')
    plt.show()

![image](http://www.playagricola.com/Kaggle/9510.png)

# The Magic Feature - UID
use the MAGIC FEATURES. 
Đầu tiên,tạo một UID sẽ giúp mô hình tìm thấy khách hàng (credit card). UID này không hoàn hảo. Nhiều giá trị UID chứa 2 hoặc nhiều khách hàng bên trong. Tuy nhiên, mô hình của này sẽ phát hiện điều này và bằng cách thêm các splits trên cây, điều này giúp tách các UID và tìm ra khách hàng riêng lẻ bên trong UID đó.

In [ ]:
X_train['day'] = X_train.TransactionDT / (24*60*60)
X_train['uid'] = X_train.card1_addr1.astype(str)+'_'+np.floor(X_train.day-X_train.D1).astype(str)

X_test['day'] = X_test.TransactionDT / (24*60*60)
X_test['uid'] = X_test.card1_addr1.astype(str)+'_'+np.floor(X_test.day-X_test.D1).astype(str)

# Tổng hợp thuộc tính - Group Aggregation Features
Để mô hình sử dụng thuộc UID mới mà ta tạo ra, cần tạo nhiều thuộc tính tổng hợp, cụ thể ta thêm 47 thuộc tính tổng hợp mới! Lưu ý rằng sau khi tổng hợp, xóa UID khỏi mô hình của mình, không sử dụng trực tiếp UID.

In [ ]:
%%time
# FREQUENCY ENCODE UID
encode_FE(X_train,X_test,['uid'])
# AGGREGATE 
encode_AG(['TransactionAmt','D4','D9','D10','D15'],['uid'],['mean','std'],fillna=True,usena=True)
# AGGREGATE
encode_AG(['C'+str(x) for x in range(1,15) if x!=3],['uid'],['mean'],X_train,X_test,fillna=True,usena=True)
# AGGREGATE
encode_AG(['M'+str(x) for x in range(1,10)],['uid'],['mean'],fillna=True,usena=True)
# AGGREGATE
encode_AG2(['P_emaildomain','dist1','DT_M','id_02','cents'], ['uid'], train_df=X_train, test_df=X_test)
# AGGREGATE
encode_AG(['C14'],['uid'],['std'],X_train,X_test,fillna=True,usena=True)
# AGGREGATE 
encode_AG2(['C13','V314'], ['uid'], train_df=X_train, test_df=X_test)
# AGGREATE 
encode_AG2(['V127','V136','V309','V307','V320'], ['uid'], train_df=X_train, test_df=X_test)
# NEW FEATURE
X_train['outsider15'] = (np.abs(X_train.D1-X_train.D15)>3).astype('int8')
X_test['outsider15'] = (np.abs(X_test.D1-X_test.D15)>3).astype('int8')
print('outsider15')

In [ ]:
cols = list( X_train.columns )
cols.remove('TransactionDT')
for c in ['D6','D7','D8','D9','D12','D13','D14']:
    cols.remove(c)
for c in ['oof','DT_M','day','uid']:
    cols.remove(c)
    
# FAILED TIME CONSISTENCY TEST
for c in ['C3','M5','id_08','id_33']:
    cols.remove(c)
for c in ['card4','id_07','id_14','id_21','id_30','id_32','id_34']:
    cols.remove(c)
for c in ['id_'+str(x) for x in range(22,28)]:
    cols.remove(c)

In [ ]:
print('NOW USING THE FOLLOWING',len(cols),'FEATURES.')
np.array(cols)

### Local Validation
Mô hình XGB sử dụng "magic work" có giá trị AUC = 0.9472 trên tập validation và 0.96 trên LB, tăng 0.01 trên tập validation và LB so với mô hình không sử dụng "magic work"


In [ ]:
# CHRIS - TRAIN 75% PREDICT 25%
idxT = X_train.index[:3*len(X_train)//4]
idxV = X_train.index[3*len(X_train)//4:]

# KONSTANTIN - TRAIN 4 SKIP 1 PREDICT 1 MONTH
#idxT = X_train.index[:417559]
#idxV = X_train.index[-89326:]

In [ ]:
if BUILD96:
    clf = xgb.XGBClassifier( 
        n_estimators=2000,
        max_depth=12, 
        learning_rate=0.02, 
        subsample=0.8,
        colsample_bytree=0.4, 
        missing=-1, 
        eval_metric='auc',
        #nthread=4,
        #tree_method='hist' 
        tree_method='gpu_hist' 
    )
    h = clf.fit(X_train.loc[idxT,cols], y_train[idxT], 
        eval_set=[(X_train.loc[idxV,cols],y_train[idxV])],
        verbose=50, early_stopping_rounds=100)

In [ ]:
if BUILD96:

    feature_imp = pd.DataFrame(sorted(zip(clf.feature_importances_,cols)), columns=['Value','Feature'])

    plt.figure(figsize=(20, 10))
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False).iloc[:50])
    plt.title('XGB96 Most Important')
    plt.tight_layout()
    plt.show()
        
    del clf, h; x=gc.collect()

### Predict test.csv

In [ ]:
if BUILD96:
    oof = np.zeros(len(X_train))
    preds = np.zeros(len(X_test))

    skf = GroupKFold(n_splits=6)
    for i, (idxT, idxV) in enumerate( skf.split(X_train, y_train, groups=X_train['DT_M']) ):
        month = X_train.iloc[idxV]['DT_M'].iloc[0]
        print('Fold',i,'withholding month',month)
        print(' rows of train =',len(idxT),'rows of holdout =',len(idxV))
        clf = xgb.XGBClassifier(
            n_estimators=5000,
            max_depth=12,
            learning_rate=0.02,
            subsample=0.8,
            colsample_bytree=0.4,
            missing=-1,
            eval_metric='auc',
            # USE CPU
            #nthread=4,
            #tree_method='hist'
            # USE GPU
            tree_method='gpu_hist' 
        )        
        h = clf.fit(X_train[cols].iloc[idxT], y_train.iloc[idxT], 
                eval_set=[(X_train[cols].iloc[idxV],y_train.iloc[idxV])],
                verbose=100, early_stopping_rounds=200)
    
        oof[idxV] += clf.predict_proba(X_train[cols].iloc[idxV])[:,1]
        preds += clf.predict_proba(X_test[cols])[:,1]/skf.n_splits
        del h, clf
        x=gc.collect()
    print('#'*20)
    print ('XGB96 OOF CV=',roc_auc_score(y_train,oof))

In [ ]:
if BUILD96:
    plt.hist(oof,bins=100)
    plt.ylim((0,5000))
    plt.title('XGB OOF')
    plt.show()

    X_train['oof'] = oof
    X_train.reset_index(inplace=True)
    X_train[['TransactionID','oof']].to_csv('oof_xgb_96.csv')
    X_train.set_index('TransactionID',drop=True,inplace=True)

## Nhìn lại quá trình làm đồ án

Sau bao ngày vất vả làm đồ án thì bây giờ đã kết thúc. Bây giờ là lúc để ngồi uống coffee và tĩnh tâm nhìn lại quá trình làm.

- Mỗi thành viên: Đã gặp những khó khăn gì? (Hay mọi chuyện đều thuận lợi)
- Mỗi thành viên: Có học được gì hữu ích? (Hay không học được gì)
- Nhóm: Nếu có thêm thời gian thì sẽ làm gì?

Phần này có sao thì bạn nói vậy thôi, chứ không phải là viết
cho có, hoặc tự chế ra để nghe cho hay.

## Tài liệu tham khảo

Để hoàn thành đồ án này, nhóm bạn đã tham khảo những tài liệu nào?